In [4]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import Output, VBox, HBox


'''
Opens a txt file output from labspec6 and reads the spectral data into the program.
Takes in the file path.
isMap, x, y (optional): tells the function whether the data to be imported is a map. If it is then the x and y range of the map must be specified for the data to import correctly.
Returns two arrays, one containing the wavenumber range (x-axis), and one containing intensity (y-axis)
'''
def importSpectrum(path, isMap = False, x = 1, y = 1):
    
    #code for importing maps
    if isMap:
        
        #open file and read the wavenumber data from the first line
        f = open(path)
        wavenumberRange = np.array(f.readline().split(), dtype = np.float32)
        
        #create empty array to hold the data
        spectGrid = np.empty([x,y,len(wavenumberRange)])
        coordGrid = np.empty([x,y, 2])
        
        #iterate through the file based on map grid size
        for i in range(x):
            for j in range(y):
                
                #read each line and split it into a list 
                temp = f.readline().split()
                
                #remove the first two entries (coordinates) from the list, then add it to the grid
                spectGrid[i,j] = np.array(temp[2:], dtype = np.float32)
                coordGrid[i,j] = np.array((temp[0], temp[1]), dtype = np.float32)
        
        #close the file and return the data
        f.close()
        return(wavenumberRange, spectGrid, coordGrid)
    
    #code for importing single spectra
    else:
        
        #open the file and read each line into a list
        with open(path) as f:
            rawData = []
            for line in f:
                rawData.append(line.split())
        
        #convert list into an array and then transpose it to switch from sets of coordinates to seperate x and y data
        rawData = np.array(rawData, dtype = np.float32)
        data = np.transpose(rawData)
        
        #close the file and return the data
        f.close()
        return(data[0], data[1])

def coordination(path, isMap = False, xRange = 1, yRange = 1):
    
    if isMap:
        x, ys, coords = importSpectrum(path, True, xRange, yRange)
        yProcessed = np.empty((yRange, xRange, len(x)))
        plot = go.FigureWidget([go.Scatter(x = x, y = ys[0][0], mode = 'lines')])
        
        red = [255, 0, 0]
        img = go.FigureWidget([go.Image(z = np.full((yRange, xRange, 3), red))])
        img.update_layout(modebar = {'remove' : ['toImage', 'zoom', 'pan', 'select', 'lasso2d', 'zoomIn', 'zoomOut', 'autoScale', 'resetScale']}, xaxis = {'fixedrange':True}, yaxis = {'fixedrange':True, 'scaleanchor':'x', 'scaleratio':1})
        
        out = Output()
        @out.capture(clear_output=True)

        def clickPixel(trace, points, selector):
            print(points)

            with plot.batch_update():
                #plot.data[0].y = yProcessed[points.ys[0], points.xs[0]]
                plot.data[0].y = ys[points.ys[0], points.xs[0]]
        
        img.data[0].on_click(clickPixel)

        dataPanel = VBox([img, out])
        dataMap = HBox([dataPanel, plot])
        return(dataMap)
        
        
    else:
        x, y = importSpectrum(path)
        baseline = findBaseline(x, y)
        #yFit, fit, finalFits = fitPeaks(x, y - baseline)
        createFig(x, [y, baseline, y - baseline])
        


#To view different sample change the number in path
#To view processed map (baseline removed) add _processed before .txt
#Processing errors occured in Sample3 positions 2,22 and 13,13 and Sample4 positions 13,19 and 20,10 so the baselines there are not removed
path = 'data/Sample4/Sample4_processed.txt'
dataMap = coordination(path, True, 25, 25)
dataMap


    'data': [{'type': 'image',
              'uid': '3cbf9b60-92c…